In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.colors import LinearSegmentedColormap
from textwrap import wrap

# Create plots directory if it doesn't exist
os.makedirs('plots', exist_ok=True)

# Load data - UPDATE THIS PATH
file_path = 'Reel Together Evaluation Form.xlsx'

# Create red-to-green colormap
colors = [(0.8, 0, 0), (1, 1, 0), (0, 0.6, 0)]  # R -> Y -> G
cmap = LinearSegmentedColormap.from_list('rg', colors, N=256)

# Get sheet names
all_sheets = pd.ExcelFile(file_path).sheet_names

# Filter target sheets
target_sheets = [sheet for sheet in all_sheets 
                if ('SUS' in sheet or 'GUESS' in sheet) 
                and 'Feedback' not in sheet]

# Determine global min/max for consistent coloring
all_means = []
for sheet in target_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df = df.drop(df.columns[0], axis=1)
    all_means.extend(df.mean().tolist())
    
global_min = min(all_means)
global_max = max(all_means)

# Create and save plots
for sheet in target_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df = df.drop(df.columns[0], axis=1)
    means = df.mean()

    # Determine normalization range and axis limits
    if 'SUS' in sheet:
        vmin, vmax = 0, 5
        xlim_right = 5
    else:
        vmin, vmax = global_min, global_max
        xlim_right = max(means) * 1.2  # 20% padding for non-SUS
    
    # Dynamic figure sizing
    fig_height = max(6, len(means) * 0.6)
    plt.figure(figsize=(12, fig_height))
    
    y_pos = np.arange(len(means))
    
    # Normalize scores for coloring
    norm_means = (means - vmin) / (vmax - vmin)
    colors = [cmap(score) for score in norm_means]
    
    bars = plt.barh(y_pos, means, align='center', color=colors, height=0.7)
    
    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=plt.gca())
    cbar.set_label('Score Range')
    
    # Labels and titles
    wrapped_labels = ['\n'.join(wrap(label, width=60)) for label in means.index]
    plt.yticks(y_pos, wrapped_labels, fontsize=8)
    
    plt.xlabel('Average Rating', fontsize=10)
    plt.title(f'Average Ratings - {sheet}', fontsize=12, pad=20)
    
    # Value labels on bars
    for i, (value, bar) in enumerate(zip(means, bars.patches)):
        plt.text(bar.get_width() + (xlim_right * 0.01),  # Small offset
                 i,
                 f'{value:.2f}',
                 va='center',
                 fontsize=8,
                 color='black')
    
    # Axis and layout
    plt.xlim(left=0, right=xlim_right)
    plt.subplots_adjust(left=0.4, right=0.85)
    plt.grid(axis='x', alpha=0.3)
    
    # Save figure
    filename = f"plots/{sheet.replace(' ', '_').replace('-', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Saved: {filename}")


Saved: plots/SUS.png
Saved: plots/GUESS___Usability_Playability.png
Saved: plots/GUESS___Narratives.png
Saved: plots/GUESS___Play_Engrossment.png
Saved: plots/GUESS___Enjoyment.png
Saved: plots/GUESS___Creative_Freedom.png
Saved: plots/GUESS___Audio_Aesthetics_.png
Saved: plots/GUESS___Personal_Gratification.png
Saved: plots/GUESS___Social_Connectivity.png
Saved: plots/GUESS___Visual_Aesthetics.png
